In [58]:
import requests
import json
from bs4 import BeautifulSoup
import time

import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import time
user = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36"
headers = {
    'User-Agent': user
}

## 球員名單

In [77]:

main_url = "https://www.cpbl.com.tw/"

url = "https://www.cpbl.com.tw/player"

res = requests.get(url, headers = headers,verify=False)
soup = BeautifulSoup(res.text, 'html.parser')

results_player= (soup.find_all(class_='PlayersList'))

print(len(results_player)) # 圖片數量


5


/Users/mac/Desktop/大學資料/OR/venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.cpbl.com.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [101]:
# print(results_player[0])

In [79]:
def get_player(results):
    team_player = results.find_all("a")
    team_title = results.find("dt").text

    return team_title, team_player

In [80]:
data = {}
for i in results_player:
    team = {}
    team_title, team_player = get_player(i)
    for p in team_player:
        player_id = p.get("href").split("=")[-1]
        team[p.text] = player_id #p.get("href")
    data[team_title] = team

In [100]:
# print(data.keys())
team_name = ["中信兄弟","味全龍","統一7-ELEVEn獅","富邦悍將","樂天桃猿"]
print(data.keys())
with open("player_list.json", "w") as f:
    json.dump(data, f, indent=4, ensure_ascii=False)

dict_keys(['中信兄弟', '味全龍', '統一7-ELEVEn獅', '富邦悍將', '樂天桃猿'])


## 分項資料

In [1]:
# data (player_list) -> need to open json player_list.json
 
import json
# Opening JSON file
f = open('player_list.json')
player_list = json.load(f)
# for i in player_list:
#     print(player_list[i])
#     break
f.close()

In [2]:
# data (player_list) -> need to open json player_list.json

player_web_list = {}
for team in player_list:
    temp = []
    for player in player_list[team]:
        web = "https://www.cpbl.com.tw/team/apart?Acnt={}".format(player_list[team][player])
        temp.append(web)
    player_web_list[team] = temp

# fight_url = ["https://www.cpbl.com.tw/team/fighting?Acnt=0000004636"]

In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select

def check_year(check):
    if "2021" not in check:
        return True

def get_data(driver):
    check = driver.find_element_by_name('year')
    check = check.text
    check = check.split("\n")
    if check_year(check):
        return True
    select = Select(driver.find_element_by_name('year'))
    select.select_by_value("2021")
    button = driver.find_element_by_css_selector(".btn")
    time.sleep(1)
    button.click()
    time.sleep(1)
    return False

def get_detail(content):
    player_detail = []
    for i in content:
        if i.text != "":
            detail = i.text
            detail = detail.split(" ")
            if len(detail) != 16:
                continue
            player_detail.append(detail)
    
    return player_detail

def web_to_data(driver,test_url):
    player_detail = []
    for url in test_url:
        print(url)
        player = {}
        driver.get(url)
        position = driver.find_element_by_class_name('pos')
        position = position.text
        position = position.split("\n")
        position = position[1]
        exit = get_data(driver)
        if exit:
            continue

        if position == "投手":
            continue
        content = driver.find_elements_by_css_selector('tr')
        detail = get_detail(content)
        name = driver.find_element_by_class_name('name')
    
        player["name"] = name.text
        player["position"] = position
        player["detail"] = detail
        player_detail.append(player)

    return player_detail


In [15]:
team = "樂天桃猿"
for i in player_web_list:
#中信兄弟 味全龍 統一7-ELEVEn獅 富邦悍將 樂天桃猿
    if i == team:
        driver = webdriver.Chrome("/Users/mac/Desktop/大學資料/OR/chromedriver")
        player_detail = web_to_data(driver,player_web_list[i])

/var/folders/8w/mfchs_0n4bd8hdwxjs7plcrc0000gn/T/ipykernel_91584/124341357.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/Users/mac/Desktop/大學資料/OR/chromedriver")


https://www.cpbl.com.tw/team/apart?Acnt=0000006718


/var/folders/8w/mfchs_0n4bd8hdwxjs7plcrc0000gn/T/ipykernel_91584/3822692903.py:40: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  position = driver.find_element_by_class_name('pos')
/var/folders/8w/mfchs_0n4bd8hdwxjs7plcrc0000gn/T/ipykernel_91584/3822692903.py:9: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  check = driver.find_element_by_name('year')


https://www.cpbl.com.tw/team/apart?Acnt=0000006228


/var/folders/8w/mfchs_0n4bd8hdwxjs7plcrc0000gn/T/ipykernel_91584/3822692903.py:14: DeprecationWarning: find_element_by_name is deprecated. Please use find_element(by=By.NAME, value=name) instead
  select = Select(driver.find_element_by_name('year'))
/var/folders/8w/mfchs_0n4bd8hdwxjs7plcrc0000gn/T/ipykernel_91584/3822692903.py:16: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  button = driver.find_element_by_css_selector(".btn")


https://www.cpbl.com.tw/team/apart?Acnt=0000003465
https://www.cpbl.com.tw/team/apart?Acnt=0000001390
https://www.cpbl.com.tw/team/apart?Acnt=0000000159


/var/folders/8w/mfchs_0n4bd8hdwxjs7plcrc0000gn/T/ipykernel_91584/3822692903.py:50: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  content = driver.find_elements_by_css_selector('tr')
/var/folders/8w/mfchs_0n4bd8hdwxjs7plcrc0000gn/T/ipykernel_91584/3822692903.py:52: DeprecationWarning: find_element_by_class_name is deprecated. Please use find_element(by=By.CLASS_NAME, value=name) instead
  name = driver.find_element_by_class_name('name')


https://www.cpbl.com.tw/team/apart?Acnt=0000001318
https://www.cpbl.com.tw/team/apart?Acnt=0000000128
https://www.cpbl.com.tw/team/apart?Acnt=0000006231
https://www.cpbl.com.tw/team/apart?Acnt=0000003563
https://www.cpbl.com.tw/team/apart?Acnt=0000005559
https://www.cpbl.com.tw/team/apart?Acnt=0000006710
https://www.cpbl.com.tw/team/apart?Acnt=0000006237
https://www.cpbl.com.tw/team/apart?Acnt=0000002286
https://www.cpbl.com.tw/team/apart?Acnt=0000000153
https://www.cpbl.com.tw/team/apart?Acnt=0000003467
https://www.cpbl.com.tw/team/apart?Acnt=0000001414
https://www.cpbl.com.tw/team/apart?Acnt=0000006235
https://www.cpbl.com.tw/team/apart?Acnt=0000003345
https://www.cpbl.com.tw/team/apart?Acnt=0000006226
https://www.cpbl.com.tw/team/apart?Acnt=0000005510
https://www.cpbl.com.tw/team/apart?Acnt=0000004634
https://www.cpbl.com.tw/team/apart?Acnt=0000006234
https://www.cpbl.com.tw/team/apart?Acnt=0000006720
https://www.cpbl.com.tw/team/apart?Acnt=0000004632
https://www.cpbl.com.tw/team/ap

In [16]:
print(len(player_detail))

23


In [17]:
import json
import pandas as pd
df = pd.DataFrame(player_detail)
print(df.head())
df.to_json("player_{}.json".format(team),orient = 'records', lines=True, force_ascii=False)

    name position                                             detail
0   成晉35     中外野手  [[分項成績, 打席, 打數, 打點, 安打, 二安, 三安, 全壘打, 壘打數, 打擊率,...
1  朱育賢85      一壘手  [[分項成績, 打席, 打數, 打點, 安打, 二安, 三安, 全壘打, 壘打數, 打擊率,...
2  余德龍36     左外野手  [[分項成績, 打席, 打數, 打點, 安打, 二安, 三安, 全壘打, 壘打數, 打擊率,...
3   林立39      三壘手  [[分項成績, 打席, 打數, 打點, 安打, 二安, 三安, 全壘打, 壘打數, 打擊率,...
4   林承飛6      游擊手  [[分項成績, 打席, 打數, 打點, 安打, 二安, 三安, 全壘打, 壘打數, 打擊率,...


## 對戰資料

In [18]:
import json
# Opening JSON file
f = open('player_list.json')
player_list = json.load(f)

f.close()

In [19]:
# data (player_list) -> need to open json player_list.json

fight_web_list = {}
for team in player_list:
    temp = []
    for player in player_list[team]:
        web = "https://www.cpbl.com.tw/team/fighting?Acnt={}".format(player_list[team][player])
        temp.append(web)
    fight_web_list[team] = temp

# fight_url = ["https://www.cpbl.com.tw/team/fighting?Acnt=0000004636"]

In [101]:
def check_year(check):
    if "2021" not in check:
        return True

def check_team(team_code,check):
    if team_code not in check:
        # print(team_code)
        return True

def get_data(driver,team_code,code):
    check = driver.find_element(by=By.NAME, value="year")
    check = check.text
    check = check.split("\n")
    if check_year(check):
        return True

    check = driver.find_element(by=By.NAME, value="fightingTeamNo") #find_element_by_name('fightingTeamNo')
    check = check.text
    check = check.split("\n")
    # print(check)
    if check_team(team_code,check):
        return True
    select = Select(driver.find_element(by=By.NAME, value="year")) #find_element_by_name('year'))
    select.select_by_value("2021")
    select2 = Select(driver.find_element(by=By.NAME, value="fightingTeamNo"))
    select2.select_by_value(code) # 味全龍AAA011 富邦悍將AEO011 統一7-ELEVEn獅ADD011 樂天桃猿AJL011
    button = driver.find_element(by=By.CSS_SELECTOR, value=".btn") #find_element_by_css_selector(".btn")
    time.sleep(1)
    button.click()
    time.sleep(1)
    return False

def get_detail(content):
    player_detail = []
    for i in content:
        if i.text != "":
            detail = i.text
            detail = detail.split(" ")
            player_detail.append(detail)
    
    return player_detail

def web_to_data(driver,test_url,team_code):
    player_detail = []
    for url in test_url:
        print(url)
        player = {}
        driver.get(url)
        position = driver.find_element(by=By.CLASS_NAME, value="pos") #find_element_by_class_name('pos')
        position = position.text
        position = position.split("\n")
        position = position[1]
        if position == "投手":
            continue
        name = driver.find_element(by=By.CLASS_NAME, value="name")
        player["name"] = name.text
        player["position"] = position

        detail_temp = []
        for team in team_code:
            exit = get_data(driver,team,team_code[team])
            if exit:
                continue

            content = driver.find_elements_by_css_selector('tr')
            detail = get_detail(content)
            detail_temp += detail
        if detail_temp == []:
            continue
        player["detail"] = detail_temp
        player_detail.append(player)

    return player_detail


In [115]:
team = "統一7-ELEVEn獅"
 # 味全龍AAA011 富邦悍將AEO011 統一7-ELEVEn獅ADD011 樂天桃猿AJL011
# '中信兄弟', '富邦悍將', '味全龍', '樂天桃猿', '統一7-ELEVEn獅'
team_dict = {'中信兄弟':"ACN011", '富邦悍將':"AEO011", '味全龍':"AAA011" , '樂天桃猿':"AJL011", '統一7-ELEVEn獅':"ADD011"}
team_code = ["AAA011","AEO011","ADD011","AJL011"]
for i in fight_web_list:
#中信兄弟 味全龍 統一7-ELEVEn獅 富邦悍將 樂天桃猿
    if i == team:
        driver = webdriver.Chrome("/Users/mac/Desktop/大學資料/OR/chromedriver")
        player_detail = web_to_data(driver,fight_web_list[i],team_dict)

/var/folders/8w/mfchs_0n4bd8hdwxjs7plcrc0000gn/T/ipykernel_91584/769797936.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/Users/mac/Desktop/大學資料/OR/chromedriver")


https://www.cpbl.com.tw/team/fighting?Acnt=0000004639
https://www.cpbl.com.tw/team/fighting?Acnt=0000003393
https://www.cpbl.com.tw/team/fighting?Acnt=0000004638
https://www.cpbl.com.tw/team/fighting?Acnt=0000005731
https://www.cpbl.com.tw/team/fighting?Acnt=0000002679
https://www.cpbl.com.tw/team/fighting?Acnt=0000003620
https://www.cpbl.com.tw/team/fighting?Acnt=0000002696


/var/folders/8w/mfchs_0n4bd8hdwxjs7plcrc0000gn/T/ipykernel_91584/113314652.py:65: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  content = driver.find_elements_by_css_selector('tr')


https://www.cpbl.com.tw/team/fighting?Acnt=0000004635
https://www.cpbl.com.tw/team/fighting?Acnt=0000006213
https://www.cpbl.com.tw/team/fighting?Acnt=0000002277
https://www.cpbl.com.tw/team/fighting?Acnt=0000005482
https://www.cpbl.com.tw/team/fighting?Acnt=0000001756
https://www.cpbl.com.tw/team/fighting?Acnt=0000004641
https://www.cpbl.com.tw/team/fighting?Acnt=0000006127
https://www.cpbl.com.tw/team/fighting?Acnt=0000006711
https://www.cpbl.com.tw/team/fighting?Acnt=0000006219
https://www.cpbl.com.tw/team/fighting?Acnt=0000002695
https://www.cpbl.com.tw/team/fighting?Acnt=0000006223
https://www.cpbl.com.tw/team/fighting?Acnt=0000002275
https://www.cpbl.com.tw/team/fighting?Acnt=0000001606
https://www.cpbl.com.tw/team/fighting?Acnt=0000000937
https://www.cpbl.com.tw/team/fighting?Acnt=0000006210
https://www.cpbl.com.tw/team/fighting?Acnt=0000001339
https://www.cpbl.com.tw/team/fighting?Acnt=0000001112
https://www.cpbl.com.tw/team/fighting?Acnt=0000002295
https://www.cpbl.com.tw/team

In [116]:
print(len(player_detail))

28


In [104]:
# print(player_detail[0]["detail"])

[['對戰球員', '打席', '打數', '打點', '安打', '二安', '三安', '全壘打', '壘打數', '打擊率', '四壞', '（故四）', '死球', '三振', '上壘率', '滾飛出局比'], ['富邦悍將\n羅力\n19', '17', '1', '6', '0', '0', '1', '9', '0.353', '2', '（0）', '0', '2', '0.421', '2.000'], ['富邦悍將\n范玉禹\n5', '4', '0', '1', '0', '0', '0', '1', '0.250', '1', '（0）', '0', '1', '0.400', '0.000'], ['富邦悍將\n陳仕朋\n11', '10', '0', '1', '0', '0', '0', '1', '0.100', '0', '（0）', '1', '1', '0.182', '1.000'], ['富邦悍將\n吳世豪\n3', '2', '0', '0', '0', '0', '0', '0', '0.000', '1', '（0）', '0', '0', '0.333', '1.000'], ['富邦悍將\n游霆崴\n20', '18', '1', '9', '1', '1', '0', '12', '0.500', '2', '（0）', '0', '0', '0.550', '1.250'], ['富邦悍將\n張瑞麟\n1', '1', '0', '0', '0', '0', '0', '0', '0.000', '0', '（0）', '0', '0', '0.000', '0.000'], ['富邦悍將\n林逸翔\n2', '2', '0', '0', '0', '0', '0', '0', '0.000', '0', '（0）', '0', '0', '0.000', '0.000'], ['富邦悍將\n江少慶\n8', '6', '2', '2', '1', '0', '0', '3', '0.333', '1', '（0）', '0', '2', '0.375', '2.000'], ['富邦悍將\n王尉永\n4', '4', '0', '3', '0', '0', '0', '3', '0.750', '0', '（

In [117]:
import json
import pandas as pd
df = pd.DataFrame(player_detail)
print(df.head())
df.to_json("fight_{}.json".format(team),orient = 'records', lines=True, force_ascii=False)

    name position                                             detail
0  江亮緯27     中外野手  [[對戰球員, 打席, 打數, 打點, 安打, 二安, 三安, 全壘打, 壘打數, 打擊率,...
1  何恆佑53      一壘手  [[對戰球員, 打席, 打數, 打點, 安打, 二安, 三安, 全壘打, 壘打數, 打擊率,...
2  吳桀睿62      一壘手  [[對戰球員, 打席, 打數, 打點, 安打, 二安, 三安, 全壘打, 壘打數, 打擊率,...
3   李丞齡9     右外野手  [[對戰球員, 打席, 打數, 打點, 安打, 二安, 三安, 全壘打, 壘打數, 打擊率,...
4   林子豪2      三壘手  [[對戰球員, 打席, 打數, 打點, 安打, 二安, 三安, 全壘打, 壘打數, 打擊率,...
